<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#package" data-toc-modified-id="package-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>package</a></span></li><li><span><a href="#function" data-toc-modified-id="function-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>function</a></span></li><li><span><a href="#main" data-toc-modified-id="main-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>main</a></span><ul class="toc-item"><li><span><a href="#get-clean-raw-data" data-toc-modified-id="get-clean-raw-data-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>get clean raw data</a></span></li><li><span><a href="#create-sia-feature" data-toc-modified-id="create-sia-feature-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>create sia feature</a></span></li><li><span><a href="#create-liwc-feature" data-toc-modified-id="create-liwc-feature-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>create liwc feature</a></span></li><li><span><a href="#save-clean-doc-with-features" data-toc-modified-id="save-clean-doc-with-features-3.4"><span class="toc-item-num">3.4&nbsp;&nbsp;</span>save clean doc with features</a></span></li></ul></li><li><span><a href="#test" data-toc-modified-id="test-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>test</a></span></li><li><span><a href="#end" data-toc-modified-id="end-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>end</a></span></li></ul></div>

In [1]:
## Pre-setting
# if one module is changed, this line will automatically reload that module
%load_ext autoreload
%autoreload 2
# display the figure in the notebook
%matplotlib inline
# To change the font size in acrobat
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42

%load_ext memory_profiler

## package

In [2]:
import os
import sys
curr_folder = os.path.dirname(__file__) if '__file__' in dir() else os.path.abspath('')
src_dir = os.path.abspath(os.path.join(curr_folder, os.pardir, 'src'))
if src_dir not in sys.path: sys.path.insert(0, src_dir)

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
## System package
import numpy as np

import matplotlib.pyplot as plt

import pandas as pd
pd.options.display.max_columns = 500
# pd.options.display.max_rows = 500

In [5]:
from nltk.tokenize import  word_tokenize
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

import json

In [6]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer


## function

## main

### get clean raw data

In [7]:
clean_data_dir = os.path.abspath(os.path.join(os.pardir, 'data', 's2021_lumen_clean_data'))
cleandoc_data_file = 's2021_05_23_02_lumen_clean_doc_data.csv'
cleandoc_data_path = os.path.join(clean_data_dir, cleandoc_data_file)

In [8]:
cd_df = pd.read_csv(cleandoc_data_path)

cd_df

,raw_text_id,raw_text,text_type,nostop_stem_doc,nostop_stem_doc_len,clean_doc,clean_doc_len,Authority or Expertise/Source Credibility,Blame/guilt,Commitment,Commitment- Call to Action,Commitment- Indignation,Emphasis,Gain framing,Liking,Loss framing,Objectivity,Reciprocation,Scarcity/Urgency/Opportunity,Social Proof,Social Proof- Admonition,Subjectivity
0,0,"""A Baker Swept By,"" by Edward Hirsch Audio: Re...",news left,baker swept edward hirsch audio read author al...,69,a baker swept by by edward hirsch audio read b...,137,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1
1,1,"""Get Out"" Won’t Have A 100% Rating On Rotten T...",news left,get rate rotten tomato ever fact jordan peel g...,99,get out won t have a rating on rotten tomatoes...,194,1,1,1,0,0,0,0,1,0,1,0,0,0,0,1
2,2,"""I wish you are still alive to see what Americ...",russian ad,wish still aliv see america turn,6,i wish you are still alive to see what america...,13,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1
3,3,"""Know Your Rights or Your Safety Is At Risk In...",russian ad,know right safeti risk interact polic shock vi...,76,know your rights or your safety is at risk in ...,146,0,1,1,1,1,1,0,0,0,1,0,0,1,1,1
4,4,"""Nancy Pelosi was drunk again today,"" begins a...",fake news,nanci pelosi drunk today begin post recent sha...,90,nancy pelosi was drunk again today begins a po...,172,1,0,0,0,0,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2993,2993,’The View’ Co-host Joy Behar: Trump Is ’The Mo...,news right,view host joy behar trump danger man aliv mond...,70,the view co host joy behar trump is the most d...,126,1,1,1,1,0,0,0,0,0,0,0,1,0,1,1
2994,2994,’Will Hillary Clinton Get Prosecuted’: Most-Go...,news right,hillari clinton get prosecut googl hillari sea...,76,will hillary clinton get prosecuted most googl...,128,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2995,2995,’You Are Amazing!’ Donald Trump Wrote To Harry...,news left,amaz donald trump wrote harri reid letter sen ...,58,you are amazing donald trump wrote to harry re...,102,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
2996,2996,“Purple Rain” Is Returning To Theaters In Trib...,news left,purpl rain return theater tribut princ purpl r...,105,purple rain is returning to theaters in tribut...,182,1,0,1,0,0,0,0,1,0,1,0,1,0,0,0


### create sia feature

In [9]:
sia = SentimentIntensityAnalyzer()

In [10]:
%%time

doc_pos = []
doc_compound = []
doc_neu = []
doc_neg = []


for tmp_d in cd_df.clean_doc:
    tmp_sia = sia.polarity_scores(tmp_d)
    
    doc_pos.append(tmp_sia['pos'])
    doc_compound.append(tmp_sia['compound'])
    doc_neu.append(tmp_sia['neu'])
    doc_neg.append(tmp_sia['neg'])
    
    

Wall time: 4.26 s


In [11]:
tmp_sia

{'neg': 0.047, 'neu': 0.817, 'pos': 0.136, 'compound': 0.8564}

In [12]:
cd_df['pos_sia'] = doc_pos
cd_df['compound_sia'] = doc_compound
cd_df['neu_sia'] = doc_neu
cd_df['neg_sia'] = doc_neg

### create liwc feature

- modified from function
    - src\obtainFeature\obtainfeature.py\obtain_liwc_feature_from_email_updateDf()

- get  **needed cleaned-up liwc feature dict**

In [13]:
liwc_feature_word_folder = os.path.abspath(os.path.join(os.pardir, 'data', 'tmp_liwc', 'processed_liwc'))
liwc_feature_word_file = 'liwc_feature_word_dic.json'

liwc_feature_word_location = os.path.join(liwc_feature_word_folder, liwc_feature_word_file)

with open(liwc_feature_word_location) as data_file:
    liwc_feature_word_dic = json.load(data_file)

In [14]:
for tmp_key in liwc_feature_word_dic.keys():
    print(tmp_key, '\t', len(liwc_feature_word_dic[tmp_key]), '\t', liwc_feature_word_dic[tmp_key][:5] )

posemo 	 464 	 ['accept', 'accepta', 'activ', 'admir', 'ador']
negemo 	 571 	 ['abandon', 'abus', 'abusi', 'ach', 'adver']
anx 	 92 	 ['afraid', 'alarm', 'anxieti', 'anxiou', 'anxious']
anger 	 176 	 ['abus', 'abusi', 'aggravat', 'aggress', 'aggressor']
sad 	 98 	 ['abandon', 'agoniz', 'agoni', 'alon', 'bereav']
reward 	 75 	 ['access', 'accru', 'accumul', 'achiev', 'achievi']
risk 	 81 	 ['abstain', 'alarm', 'apprehen', 'apprenhen', 'avers']
time 	 251 	 ['abrupt', 'after', 'aftermath', 'afternoon', 'afterthought']
money 	 167 	 ['account', 'accru', 'afford', 'atm', 'auction']


In [15]:
ps = PorterStemmer()

liwc_feature_list = list(liwc_feature_word_dic.keys())

liwc_feature_count_list_dict = {}
for i in liwc_feature_list:
    liwc_feature_count_list_dict[i] = []

In [16]:
%%time


for tmp_string in cd_df.clean_doc:

    tmp_liwc_feature_count_dict = {}
    for i in liwc_feature_list:
        tmp_liwc_feature_count_dict[i] = 0

    for tmp_word in tmp_string.split():
        tmp_word_stem = ps.stem(tmp_word)

        for tmp_liwc_feature in liwc_feature_list:
            if tmp_word_stem in liwc_feature_word_dic[tmp_liwc_feature]:
                tmp_liwc_feature_count_dict[tmp_liwc_feature] += 1

    for i in liwc_feature_list:
        liwc_feature_count_list_dict[i].append(tmp_liwc_feature_count_dict[i])

Wall time: 21.7 s


In [17]:
for i in liwc_feature_list:
    print(i, len(liwc_feature_count_list_dict[i]))

posemo 2998
negemo 2998
anx 2998
anger 2998
sad 2998
reward 2998
risk 2998
time 2998
money 2998


In [18]:
for i in liwc_feature_list:
    cd_df['%s_liwc'%i] = liwc_feature_count_list_dict[i]

### save clean doc with features

In [19]:
list(cd_df.columns)

['raw_text_id',
 'raw_text',
 'text_type',
 'nostop_stem_doc',
 'nostop_stem_doc_len',
 'clean_doc',
 'clean_doc_len',
 'Authority or Expertise/Source Credibility',
 'Blame/guilt',
 'Commitment',
 'Commitment- Call to Action',
 'Commitment- Indignation',
 'Emphasis',
 'Gain framing',
 'Liking',
 'Loss framing',
 'Objectivity',
 'Reciprocation',
 'Scarcity/Urgency/Opportunity',
 'Social Proof',
 'Social Proof- Admonition',
 'Subjectivity',
 'pos_sia',
 'compound_sia',
 'neu_sia',
 'neg_sia',
 'posemo_liwc',
 'negemo_liwc',
 'anx_liwc',
 'anger_liwc',
 'sad_liwc',
 'reward_liwc',
 'risk_liwc',
 'time_liwc',
 'money_liwc']

In [20]:
columns = [
    'raw_text_id',
    'raw_text',
    'text_type',
    'nostop_stem_doc',
    'nostop_stem_doc_len',
    'clean_doc',
    'clean_doc_len',

    'pos_sia',
    'compound_sia',
    'neu_sia',
    'neg_sia',

    'posemo_liwc',
    'negemo_liwc',
    'anx_liwc',
    'anger_liwc',
    'sad_liwc',
    'reward_liwc',
    'risk_liwc',
    'time_liwc',
    'money_liwc',

    'Authority or Expertise/Source Credibility',
    'Blame/guilt',
    'Commitment',
    'Commitment- Call to Action',
    'Commitment- Indignation',
    'Emphasis',
    'Gain framing',
    'Liking',
    'Loss framing',
    'Objectivity',
    'Reciprocation',
    'Scarcity/Urgency/Opportunity',
    'Social Proof',
    'Social Proof- Admonition',
    'Subjectivity',
]

In [21]:
cd_df = cd_df[columns]

In [22]:

cd_sia_liwc_file = 's2021_05_23_03_lumen_clean_doc_sia_liwc.csv'
cd_sia_liwc_path = os.path.join(clean_data_dir, cd_sia_liwc_file)

In [23]:
cd_df.to_csv(cd_sia_liwc_path, index=False)

## test

In [24]:
1

1

## end

In [25]:
1

1